In [1]:
from gwdataset import GW_Dataset
from tensorflow import keras
from tensorflow.keras import layers

/opt/miniconda3/envs/keras2/lib/python3.7/site-packages/gwtools/rotations.py:63: UserWarning: Could not import GWFrames, needed for rotations module
  _warnings.warn("Could not import GWFrames, needed for rotations module")
/opt/miniconda3/envs/keras2/lib/python3.7/site-packages/gwtools/__init__.py:11: UserWarning: Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.
  _warnings.warn("Could not import rotations, decompositions, or fitfuncs. These are not needed by GWSurrogate.")


setting __package__ to gwsurrogate.new so relative imports work
__name__ = gwsurrogate.new.spline_evaluation
__package__= gwsurrogate.new
setting __package__ to gwsurrogate.new so relative imports work
setting __package__ to gwsurrogate.new so relative imports work


2021-08-23 08:32:01.404996: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [3]:
Dataset= GW_Dataset(load_dataset = True)

In [4]:
Dataset.load_waveform(data_fn='waveform_dataset_4096Hz.hdf5')

In [10]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


"""
The main part of our model is now complete. We can stack multiple of those
`transformer_encoder` blocks and we can also proceed to add the final
Multi-Layer Perceptron classification head. Apart from a stack of `Dense`
layers, we need to reduce the output tensor of the `TransformerEncoder` part of
our model down to a vector of features for each data point in the current
batch. A common way to achieve this is to use a pooling layer. For
this example, a `GlobalAveragePooling1D` layer is sufficient.
"""


def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(4096, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [11]:
input_shape = [4096,1]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
)
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4096, 1)]    0                                            
__________________________________________________________________________________________________
multi_head_attention_4 (MultiHe (None, 4096, 1)      7169        input_2[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 4096, 1)      0           multi_head_attention_4[0][0]     
__________________________________________________________________________________________________
layer_normalization_8 (LayerNor (None, 4096, 1)      2           dropout_9[0][0]            

In [7]:
for key in Dataset.train_waveform.keys():
    Dataset.train_waveform[key] = Dataset.train_waveform[key].reshape((Dataset.train_waveform[key].shape[0], Dataset.train_waveform[key].shape[1], 1))

In [8]:
for key in Dataset.train_waveform.keys():
    Dataset.test_waveform[key] = Dataset.test_waveform[key].reshape((Dataset.test_waveform[key].shape[0], Dataset.test_waveform[key].shape[1], 1))

In [12]:
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

history = model.fit(
    Dataset.train_waveform['noisy'],
    Dataset.train_waveform['clean'],
    validation_split=0.2,
    epochs=200,
    batch_size=1,
    callbacks=callbacks,
)

Epoch 1/200


2021-08-23 08:36:14.062875: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-23 08:36:14.364516: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-08-23 08:36:14.388667: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-23 08:36:14.687643: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2021-08-23 08:36:14.976122: W tensorflow/stream_executor/gpu/asm_compiler.cc:99] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You do not need to update to CUDA 9.2.88; cherry-picking the ptxas binary is sufficient.


1898/1898 [==============================] - 355s 186ms/step - loss: 0.0738 - val_loss: 0.0897
Epoch 2/200
1898/1898 [==============================] - 351s 185ms/step - loss: 0.0738 - val_loss: 0.0897
Epoch 3/200
1898/1898 [==============================] - 351s 185ms/step - loss: 0.0738 - val_loss: 0.0897
Epoch 4/200
1898/1898 [==============================] - 351s 185ms/step - loss: 0.0738 - val_loss: 0.0897
Epoch 5/200
1546/1898 [=======================>......] - ETA: 1:00 - loss: 0.0738

KeyboardInterrupt: 